In [215]:
#import libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [216]:
#read people data
people_df = pd.read_csv("neuro_people.tsv", sep = "\t")

In [217]:
#read relation data
rel_df = pd.read_csv("neuro_relation.tsv", sep = "\t")
rel_df.head()

,cid,pid1,pid2,relation,location,locid,department,startdate,stopdate,addedby,dateadded
0,2,2,3,1,"University of California, Berkeley",312,NaN,2000-00-00,2005-00-00,david,2005-01-15 17:33:57
1,3,4,3,2,"University of California, Berkeley",312,\N,2003-00-00,2006-00-00,david,2005-01-15 17:35:17
2,5,6,3,1,"University of California, Berkeley",312,NaN,2002-00-00,2008-00-00,david,2005-01-15 19:02:54
3,6,18761,9,1,"University of California, Berkeley",312,\N,0000-00-00,1984-00-00,david,2005-01-15 19:06:16
4,7,10,16,2,"Washington University, Saint Louis",497,\N,0000-00-00,0000-00-00,david,2005-01-15 19:06:58


* It seems that "\N" and NaN are used at the same time so a change is required. 

In [218]:
#explorring missing 

# \N --> NaN change
people_df = people_df.replace("\\N", float('nan'))
rel_df = rel_df.replace("\\N", float('nan'))

people_df = people_df.replace(float('nan'), "-9")
rel_df = rel_df.replace(float('nan'), "-9")

people_df = people_df.replace('', "-9")
rel_df = rel_df.replace('', "-9")

people_df.head()

,pid,firstname,middlename,lastname,degrees,location,locid,area,majorarea,award,orcid_id,homepage,addedby,dateadded
0,1,Stephen,V.,David,Ph.D.,Oregon Health and Science University,226,"auditory system, attention","neuro,csd,bme",-9,0000-0003-4135-3104,http://hearingbrain.org/,david,2005-01-14 15:50:37
1,3,Jack,L.,Gallant,-9,"University of California, Berkeley",312,Systems,"neuro,psych",-9,-9,http://gallantlab.org,david,2005-01-14 15:51:51
2,5,Kendrick,Norris,Kay,Ph.D.,"University of Minnesota, Twin Cities",406,"visual system, fMRI, computational methods, ne...",neuro,-9,-9,http://cvnlab.net,david,2005-01-15 18:20:41
3,6,Ryan,-9,Prenger,-9,Lawrence Livermore National Laboratory,1480,visual system,neuro,-9,-9,-9,david,2005-01-15 18:23:32
4,8,Kathleen,A.,Hansen,-9,National Institute of Mental Health,214,-9,neuro,-9,-9,-9,hayden,2005-01-15 18:38:58


In [219]:
rel_df.head()

,cid,pid1,pid2,relation,location,locid,department,startdate,stopdate,addedby,dateadded
0,2,2,3,1,"University of California, Berkeley",312,-9,2000-00-00,2005-00-00,david,2005-01-15 17:33:57
1,3,4,3,2,"University of California, Berkeley",312,-9,2003-00-00,2006-00-00,david,2005-01-15 17:35:17
2,5,6,3,1,"University of California, Berkeley",312,-9,2002-00-00,2008-00-00,david,2005-01-15 19:02:54
3,6,18761,9,1,"University of California, Berkeley",312,-9,0000-00-00,1984-00-00,david,2005-01-15 19:06:16
4,7,10,16,2,"Washington University, Saint Louis",497,-9,0000-00-00,0000-00-00,david,2005-01-15 19:06:58


In [220]:
# Counting by columns in people_df: degrees, locid, area, award
loc_string_cnt = people_df['location'].value_counts()
loc_id_cnt = people_df['locid'].value_counts()

loc_string_cnt.values
print(loc_id_cnt.values)

#2982 is the counting for the NaN
loc_id_cnt_re = loc_id_cnt[1:]

[2982 1423 1384 ...    1    1    1]


In [221]:
print(len(loc_id_cnt_re))
print(len(loc_string_cnt))

10166
10270


some string location does not match to locid. The below tried to find out which locid has more than one string values.

In [222]:
#for people
def double_entry_check(id_col, str_col, df):
    check_dict = {}
    for index, row in df.iterrows():
        loc_str = row[str_col]
        loc_id = row[id_col]
        if loc_id not in check_dict.keys():
            check_dict[loc_id] = set([loc_str])
        elif loc_id in check_dict.keys():
            existing_set = check_dict.get(loc_id)
            existing_set.update([loc_str])
            check_dict[loc_id] = existing_set

    for pair in check_dict.items():
        if len(pair[1]) > 1:
            print(pair)

double_entry_check('locid', 'location', people_df)

('1', {'Array', '-9'})
('227', {'Osaka University', 'Osaka University '})
('472', {'University of Würzburg ', 'University of Würzburg', 'University of Wurzburg'})
('422', {'University of Oregon', 'University of Oregon '})
('26', {'Boston College ', 'Boston College'})
('411', {'University of Newcastle', 'University of newcastle '})
('385', {'University of Illinois', 'University of Illinois '})
('240', {'Array', "Queen's University ", "Queen's University"})
('361', {'University of Bristol ', 'University of Bristol'})
('600', {'Dartmouth College ', 'Dartmouth College'})
('151', {'Max Planck Institute for Biological Cybernetics', 'Max Planck Institute for biological Cybernetics'})
('15961', {'Icahn School of Medicine at Mount Sinai', 'Icahn School of Medicine at Mount Sinai '})
('614', {'University of Texas Health Science Center at Houston', 'University of Texas Health Science Center at Houston '})
('467', {'University of Western Ontario', 'University of Western Ontario '})
('325', {'Umea 

In [223]:
#for rel
double_entry_check('locid', 'location', rel_df)

('393', {'University of Leipzig', 'University of Leipzig '})
('407', {'University of Missouri', 'University of Missouri '})
('59', {'Duke University ', 'Duke University'})
('600', {'Dartmouth College ', 'Dartmouth College'})
('385', {'University of Illinois', 'University of Illinois '})
('270', {'SISSA', 'SISSA '})
('15961', {'Icahn School of Medicine at Mount Sinai', 'Mount Sinai School of Medicine', 'Icahn School of Medicine at Mount Sinai '})
('467', {'University of Western Ontario', 'University of Western Ontario '})
('6428', {'Delft University of Technology', 'Delft University of Technology '})
('325', {'Ume?University', 'Umea University'})
('103', {'INSERM', 'INSERM '})
('281', {'Stockholm University ', 'Stockholm University'})
('240', {"Queen's University ", "Queen's University"})
('227', {'Osaka University', 'Osaka University '})
('123', {' Karolinska Institute', 'Karolinska Institute'})
('422', {'University of Oregon', 'University of Oregon '})
('361', {'University of Bristol 

1) The spaces on the each end side raise problem;

2) the lower case or the capital.

In [224]:
people_df['location'] = people_df['location'].str.rstrip(" ").str.lstrip(" ").str.lower()
rel_df['location'] = rel_df['location'].str.rstrip(" ").str.lstrip(" ").str.lower()

In [225]:
double_entry_check('locid', 'location', people_df)

('1', {'array', '-9'})
('472', {'university of wurzburg', 'university of würzburg'})
('240', {'array', "queen's university"})
('325', {'umea university', 'umeå university'})
('344', {'universite laval', 'université laval'})
('625', {'array', 'university of bonn'})
('1052', {'university of erlangen-nürnberg', 'university of erlangen-nurnberg'})
('51', {'collège de france', 'college de france'})
('5523', {'university of texas at el paso', 'ut el paso'})
('2401', {'university of jyväskylä', 'university of jyvaskyla'})
('5693', {'institute of neuroinformatics, eth zurich', 'institute of neuroinformatics, eth zürich'})
('6155', {'university of münster', 'university of munster'})
('8228', {'linköping university', 'linkoping university'})
('-9', {'array', '-9'})


In [226]:
double_entry_check('locid', 'location', rel_df)

('15961', {'mount sinai school of medicine', 'icahn school of medicine at mount sinai'})
('325', {'umea university', 'ume?university'})
('344', {'universite laval', 'universit?laval'})
('472', {'university of wurzburg', 'university of w?zburg'})
('1052', {'university of erlangen-n?nberg', 'university of erlangen-nurnberg'})
('51', {'coll?e de france', 'college de france'})
('-9', {'university of jyv?kyl?\t2401"', 'instituto federal de educa豫o, ci?cia e tecnologia do paran?\t27586"', 'istituto superiore di sanit?\t18837"', 'universidade federal do paran?\t5731"', 'aix-marseille universit?\t17653"', 'columbia university?\t23687"', 'charit?\t4158"', 'universiti utara malaysia?\t23690"', 'mirail universit?\t16314"', 'h?ital robert debr?\t15298"', 'universidade federal do cear?\t5730"', 'humbolt university berlin, cellular neurophysiology and pathophysiology, charit?\t2630"', 'aix marseille universit?\t17130"', 'ufpa - universidade federal do par?\t28553"', 'universidade federal do par?\t37

Non-English alphabet creates the problem. But the locid captures the unique location. Later we can change the name of the universities 

The counting of degree types/the number of universities and area.

In [227]:
degrees_type = people_df['degrees'].value_counts()
len(degrees_type)

1218

In [230]:
degrees_type

-9                                                                                                                  59674
Ph.D.                                                                                                               46345
Psy.D.                                                                                                               4881
PhD                                                                                                                  2991
Ed.D.                                                                                                                1223
MD                                                                                                                   1105
Ph.D                                                                                                                  560
M.D.                                                                                                                  408
MD, PhD                 

Needs cleaning... 1218 types??????

- Variations for "MD PhD" and "PhD" --> seems to be two main patterns. 

- some contains their school info.

- Psy.D, == D.Ps == D.Psy. The three are the same

- D.Ed. == Ed.D.

- MSc = M.S. = M.Sc.  = MS

- DVM == Doctor of Veterinary Medicine

- DBA == Doctor of Business Administration


I decided to erase "." first and how this narrow down the variation

In [233]:
#erase "." and make all string upper case
degree_cleaned= people_df['degrees'].str.replace(".", "").str.rstrip(" ").str.upper()

#PH D --> PHD
degree_cleaned = degree_cleaned.str.replace("PH D", "PHD")
print(degree_cleaned.value_counts().head(20))
print(degree_cleaned.value_counts(normalize = True).head(20))

-9         59674
PHD        50141
PSYD        4891
MD          1538
EDD         1228
MD, PHD      486
MSC          343
BS           259
MD PHD       246
MS           225
BA           140
DMA          120
MA           115
DR           106
BSC           99
DBA           93
DED           85
EDS           71
DPS           63
MD/PHD        54
Name: degrees, dtype: int64
-9         0.489850
PHD        0.411596
PSYD       0.040149
MD         0.012625
EDD        0.010080
MD, PHD    0.003989
MSC        0.002816
BS         0.002126
MD PHD     0.002019
MS         0.001847
BA         0.001149
DMA        0.000985
MA         0.000944
DR         0.000870
BSC        0.000813
DBA        0.000763
DED        0.000698
EDS        0.000583
DPS        0.000517
MD/PHD     0.000443
Name: degrees, dtype: float64


The majorities are mssing or PhD...? some are pure MD. and there are some MD/PhD, even though the proportion is not big. TMD/PhD might need a special attention so further cleaning may be required after the target sample is fixed but not now.

In [234]:
#award?